In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,22701
2,Huelva,Confirmados PDIA 14 días,5008
3,Huelva,Tasa PDIA 14 días,"975,894927606836"
4,Huelva,Confirmados PDIA 7 días,1913
5,Huelva,Total Confirmados,22929
6,Huelva,Curados,11906
7,Huelva,Fallecidos,197


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  22701.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6027.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 342 personas en los últimos 7 días 

Un positivo PCR cada 117 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,22701.0,1913.0,5008.0,513170.0,372.780950,975.894928,299.0
Huelva-Costa,13783.0,1103.0,3042.0,289548.0,380.938566,1050.603009,202.0
Huelva (capital),6027.0,420.0,1226.0,143837.0,291.997191,852.353706,80.0
Condado-Campiña,6254.0,615.0,1471.0,156231.0,393.647868,941.554493,64.0
Ayamonte,1087.0,123.0,335.0,21104.0,582.827900,1587.376801,28.0
Isla Cristina,1862.0,120.0,381.0,21393.0,560.931146,1780.956388,24.0
Aljaraque,886.0,61.0,168.0,21474.0,284.064450,782.341436,21.0
Lepe,1820.0,158.0,401.0,27880.0,566.714491,1438.307030,17.0
Rociana del Condado,276.0,47.0,105.0,7939.0,592.014108,1322.584708,10.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Berrocal,17.0,3.0,9.0,303.0,990.099010,2970.297030,1.0
Villalba del Alcor,422.0,24.0,91.0,3366.0,713.012478,2703.505645,0.0
Paymogo,67.0,12.0,31.0,1162.0,1032.702238,2667.814114,0.0
Cabezas Rubias,28.0,7.0,14.0,706.0,991.501416,1983.002833,1.0
San Bartolomé de la Torre,184.0,27.0,74.0,3761.0,717.894177,1967.561819,0.0
Villablanca,135.0,32.0,53.0,2885.0,1109.185442,1837.088388,7.0
Isla Cristina,1862.0,120.0,381.0,21393.0,560.931146,1780.956388,24.0
Palma del Condado (La),933.0,78.0,179.0,10801.0,722.155356,1657.253958,5.0
Ayamonte,1087.0,123.0,335.0,21104.0,582.827900,1587.376801,28.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cortegana,198.0,1.0,28.0,4602.0,21.729683,608.431117,0.0,0.035714
Encinasola,116.0,1.0,11.0,1310.0,76.335878,839.694656,0.0,0.090909
Nerva,145.0,6.0,36.0,5169.0,116.076611,696.459663,0.0,0.166667
Zalamea la Real,82.0,5.0,25.0,3054.0,163.719712,818.598559,0.0,0.200000
Villarrasa,126.0,3.0,14.0,2211.0,135.685210,633.197648,0.0,0.214286
Aracena,444.0,13.0,52.0,8255.0,157.480315,629.921260,1.0,0.250000
Beas,142.0,9.0,36.0,4341.0,207.325501,829.302004,3.0,0.250000
Punta Umbría,598.0,48.0,185.0,15355.0,312.601758,1204.819277,10.0,0.259459
Villalba del Alcor,422.0,24.0,91.0,3366.0,713.012478,2703.505645,0.0,0.263736
